In [10]:
### PREPARATION ###

import pandas as pd
import numpy as np

#Loading Datasets

hdfc = pd.read_csv (r'C:\Users\MOHIT.K\Desktop\STock Market Prediction Literature\Stock trend Prediction\HDFC_NSE.csv')
n_50 = pd.read_csv (r'C:\Users\MOHIT.K\Desktop\STock Market Prediction Literature\Stock trend Prediction\NIFTY_50.csv')
n_bank = pd.read_csv (r'C:\Users\MOHIT.K\Desktop\STock Market Prediction Literature\Stock trend Prediction\NIFTY_BANK.csv')

#Renaming Columns and Dropping Date, Volume and Close Columns 

n_50.rename(columns = {'Date':'N50_Date', 'Open':'N50_Open', 'High':'N50_High', 'Low':'N50_Low', 'Close':'N50_Close', 'Adj Close': 'N50_Adj Close', 'Volume': 'N50_Volume'}, inplace = True)
n_bank.rename(columns = {'Date':'NB_Date','Open':'NB_Open', 'High':'NB_High', 'Low':'NB_Low', 'Close':'NB_Close', 'Adj Close': 'NB_Adj Close', 'Volume': 'NB_Volume'}, inplace = True)

HDFC = hdfc.drop(['Date', 'Close'], axis = 1)
Nifty_50 = n_50.drop(['N50_Date', 'N50_Volume', 'N50_Close', 'N50_Open', 'N50_High', 'N50_Low'], axis=1)
Nifty_Bank = n_bank.drop(['NB_Date', 'NB_Volume', 'NB_Close', 'NB_Open', 'NB_High', 'NB_Low'], axis=1)

#Merging the datasets and deleting rows with any null values to create Project Dataset

HDFC_Working = pd.concat([HDFC, Nifty_50, Nifty_Bank], axis = 1)
HDFC_Working = HDFC_Working.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

HDFC_Working.reset_index(drop=True, inplace=True)

#Adding a Direction column for classification

HDFC_Working_C = HDFC_Working.copy()

HDFC_Working_C["Sign"] =  HDFC_Working_C["Adj Close"].diff(1)
HDFC_Working_C.iat[0, 7] = 0

HDFC_Working_C["Direction"] = ""

i = 0
for j in range(2217):

    if(HDFC_Working_C.iat[i, 7] >= 0):
        HDFC_Working_C.iat[i, 8] = 1
    else:
        HDFC_Working_C.iat[i, 8]= 0
    i = i + 1

HDFC_Working_C = HDFC_Working_C.drop(['Sign'], axis = 1)

#Creating Train and Test sets for Regression

HDFC_Working_Train = HDFC_Working[ : 1774]
HDFC_Working_Test = HDFC_Working[1774 : ]

#Creating Train and Test sets for Classifiaction

HDFC_Working_C_train=HDFC_Working_C[ : 1774]
HDFC_Working_C_test=HDFC_Working_C[1774 : ]

,Open,High,Low,Adj Close,Volume,N50_Adj Close,NB_Adj Close
0,237.000000,239.964996,235.039993,223.205643,5398390.0,6157.600098,11855.750000
1,238.649994,238.800003,233.300003,219.008545,8284130.0,6146.350098,11564.049805
2,234.970001,234.970001,229.750000,215.521133,7825920.0,6079.799805,11305.450195
3,231.600006,234.264999,230.899994,217.346527,5435090.0,6048.250000,11186.799805
4,232.619995,232.619995,226.100006,211.902969,7948010.0,5904.600098,11053.349609


In [29]:
###Linear Regressor###

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

HDFC_Targets = HDFC_Working_Train["Adj Close"].copy()
HDFC_Linear =  HDFC_Working_Train.drop("Adj Close", axis=1)

lin_reg = LinearRegression()

#Cross Validation

scores = cross_val_score(lin_reg, HDFC_Linear, HDFC_Targets, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(rmse_scores)

Scores: [3.31192413 1.5844025  2.13296101 2.2205876  2.17349932 2.68644882
 3.07673462 3.61978007 3.78477728 4.8097414 ]
Mean: 2.9400856751874604
Standard deviation: 0.9212507010126924


In [24]:
###Decision Tree Regressor##$

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score


HDFC_Targets = HDFC_Working_Train["Adj Close"].copy()
HDFC_DT =  HDFC_Working_Train.drop("Adj Close", axis=1)

tree_reg = DecisionTreeRegressor()

#Cross Validation

scores = cross_val_score(tree_reg, HDFC_DT, HDFC_Targets, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(rmse_scores)

Scores: [ 3.20549442  9.59617263  6.74705175  4.58211489 24.83026496 47.80976861
 37.39001358 45.32447671  9.9676972  25.17077773]
Mean: 21.462383248633106
Standard deviation: 16.27913276998717


In [28]:
###Random Forest Regressor###

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

HDFC_Targets = HDFC_Working_Train["Adj Close"].copy()
HDFC_RF =  HDFC_Working_Train.drop("Adj Close", axis=1)

forest_reg = RandomForestRegressor()

#Cross Validation

scores = cross_val_score(forest_reg, HDFC_RF, HDFC_Targets, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(rmse_scores)

#Though the scores here are better than Decision Tree, they are worse than Linear Regression. 
#Is it so because the Random Forest Regressor is still overfitting the data?

Scores: [ 3.64397953  4.83167232  5.78585475  3.49308556 11.58060523 27.25211368
 47.4246444  35.49944333  7.84587994 26.93143774]
Mean: 17.428871649012443
Standard deviation: 14.891613069630766


In [30]:
### SVR ###

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR

HDFC_Targets = HDFC_Working_Train["Adj Close"].copy()
HDFC_SVR =  HDFC_Working_Train.drop("Adj Close", axis=1)

svm_reg = SVR()


#Cross Validation

scores = cross_val_score(svm_reg, HDFC_SVR, HDFC_Targets, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(rmse_scores)

#No idea why SVR is overfitting the data so badly

Scores: [181.22790083 180.8573404  146.31921916  99.38778745 115.82269603
  74.81706132 155.21626852 427.72404991 579.343703   694.26490977]
Mean: 265.4980936377505
Standard deviation: 208.72065833396718


In [ ]:
### SVR with GRID SEARCH ###

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

HDFC_Targets = HDFC_Working_Train["Adj Close"].copy()
HDFC_SVR =  HDFC_Working_Train.drop("Adj Close", axis=1)

svm_reg = SVR()

#Grid Search

param_grid ={'C': [0.1, 1],  
              'gamma': [1, 0.1], 
              'gamma':['scale', 'auto'],
              'kernel': ['linear', 'rbf']}

grid_search = GridSearchCV(svm_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(HDFC_SVR, HDFC_Targets)

grid_search.best_params_


In [19]:
### SGD Classifier ###

import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier

HDFC_Binary_Targets = HDFC_Working_C_train["Direction"].copy()
HDFC_BinaryClass = HDFC_Working_C_train.drop("Direction", axis = 1)


sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(HDFC_BinaryClass, HDFC_Binary_Targets)

ValueError: Unknown label type: (array([0, 1], dtype=object),)